Load in all required libraries

In [ ]:
import pandas as pd
import xarray as xr
import rioxarray
import netCDF4
import numpy as np
from shapely.geometry import mapping
import geopandas as gpd
from metsim import MetSim
from core.utils import Load_Environment
cur_dir = Load_Environment().cur_dir

In [ ]:
def reformat_data(pr, varname):
    '''
    Function to restructure netcdf file for metsim
    pr : xr.Dataset
    varname: str
    '''
    coords = {'time': pr.time, 'lat': pr.lat, 'lon': pr.lon}
    dims = ('time','lat','lon')
    met_data = xr.Dataset(coords=coords)
    met_data[varname] = xr.DataArray(data=pr[varname], coords=coords, dims = dims,name = varname)
    return met_data


In [ ]:
def process_forcing_data(cur_dir, year, gdf):
    
    tmin = xr.open_dataset(cur_dir.joinpath('input',f'tmmn_{year}.nc'))
    tmin = tmin.rename_vars({'air_temperature':'t_min'})
    tmin = tmin.rename_dims({'day':'time'})
    tmin['time'] = tmin['day']
    tmin = tmin.drop('day')
    tmin = tmin.drop('crs')
    tmin = tmin.rio.write_crs("EPSG:4326")
    tmin = tmin.rio.clip(gdf.geometry.apply(mapping), gdf.crs, drop=True, all_touched=True)
    tmin = tmin.drop('crs')
    tmin = reformat_data(tmin, 't_min')

    tmax = xr.open_dataset(cur_dir.joinpath('input',f'tmmx_{year}.nc'))
    tmax = tmax.rename_vars({'air_temperature':'t_max'})
    tmax = tmax.rename_dims({'day':'time'})
    tmax['time'] = tmax['day']
    tmax = tmax.drop('day')
    tmax = tmax.drop('crs')
    tmax = tmax.rio.write_crs("EPSG:4326")
    tmax = tmax.rio.clip(gdf.geometry.apply(mapping), gdf.crs, drop=True, all_touched=True)
    tmax = tmax.drop('crs')
    tmax = reformat_data(tmax, 't_max')
    
    pr = xr.open_dataset(cur_dir.joinpath('input',f'pr_{year}.nc'))
    pr = pr.rename_vars({'precipitation_amount':'prec'})
    pr = pr.rename_dims({'day':'time'})
    pr['time'] = pr['day']
    pr = pr.drop('day')
    pr = pr.drop('crs')
    pr = pr.rio.write_crs("EPSG:4326")
    pr = pr.rio.clip(gdf.geometry.apply(mapping), gdf.crs, drop=True, all_touched=True)
    pr = pr.drop('crs')
    pr = reformat_data(pr, 'prec')

    met_data = xr.merge([pr, tmax, tmin])
    met_data.to_netcdf(cur_dir.joinpath('input',f'chinle_forcing_{year}.nc'))

    return met_data
    
    

In [ ]:
def process_domain_data(cur_dir, met_data, gdf):
    met_data = met_data.rio.write_crs('EPSG:4326')

    domain = xr.open_dataset(cur_dir.joinpath('input','domain.CONUS_MX.L2015.nc'))
    domain = domain.rio.write_crs("EPSG:4326")

    #T_pk
    a = domain.t_pk.rio.reproject_match(met_data)
    a = a.rename(x = 'lon', y = 'lat')
    a = a.rio.clip(gdf.geometry.apply(mapping), gdf.crs, drop=True, all_touched=True)

    #DUR
    b = domain.dur.rio.reproject_match(met_data)
    b = b.rename(x = 'lon', y = 'lat')
    b = b.rio.clip(gdf.geometry.apply(mapping), gdf.crs, drop=True, all_touched=True)

    #elev
    c = domain.elev.rio.reproject_match(met_data)
    c = c.rename(x = 'lon', y = 'lat')
    c = c.rio.clip(gdf.geometry.apply(mapping), gdf.crs, drop=True, all_touched=True)

    #mask
    d = domain.mask.rio.reproject_match(met_data)
    d = d.rename(x = 'lon', y = 'lat')
    d = d.rio.clip(gdf.geometry.apply(mapping), gdf.crs, drop=True, all_touched=True)


    e = xr.merge([a,b,c,d])
    e.to_netcdf(cur_dir.joinpath('input','chinle_domain.nc'))

    return e

In [ ]:
def process_state_data(cur_dir, year, gdf):
    '''
    cur_dir: 
    year
    gdf
    '''

    tmax = xr.open_dataset(cur_dir.joinpath('input',f'tmmx_{year-1}.nc'))
    tmax = tmax.sel(day=slice(f'{year-1}-10-03',f'{year-1}-12-31'))
    tmax = tmax.rename_vars({'air_temperature':'t_max'})
    tmax = tmax.rename_dims({'day':'time'})
    tmax['time'] = tmax['day']
    tmax = tmax.drop('day')
    tmax = tmax.drop('crs')
    tmax = tmax.rio.write_crs("EPSG:4326")
    tmax = tmax.rio.clip(gdf.geometry.apply(mapping), gdf.crs, drop=True, all_touched=True)
    tmax = tmax.drop('crs')
    tmax = reformat_data(tmax, 't_max')

    tmin = xr.open_dataset(cur_dir.joinpath('input',f'tmmn_{year-1}.nc'))
    tmin = tmin.sel(day=slice(f'{year-1}-10-03',f'{year-1}-12-31'))
    tmin = tmin.rename_vars({'air_temperature':'t_min'})
    tmin = tmin.rename_dims({'day':'time'})
    tmin['time'] = tmin['day']
    tmin = tmin.drop('day')
    tmin = tmin.drop('crs')
    tmin = tmin.rio.write_crs("EPSG:4326")
    tmin = tmin.rio.clip(gdf.geometry.apply(mapping), gdf.crs, drop=True, all_touched=True)
    tmin = tmin.drop('crs')
    tmin = reformat_data(tmin, 't_min')

    pr = xr.open_dataset(cur_dir.joinpath('input',f'pr_{year-1}.nc'))
    pr = pr.sel(day=slice(f'{year-1}-10-03',f'{year-1}-12-31'))
    pr = pr.rename_vars({'precipitation_amount':'prec'})
    pr = pr.rename_dims({'day':'time'})
    pr['time'] = pr['day']
    pr = pr.drop('day')
    pr = pr.drop('crs')
    pr = pr.rio.write_crs("EPSG:4326")
    pr = pr.rio.clip(gdf.geometry.apply(mapping), gdf.crs, drop=True, all_touched=True)
    pr = pr.drop('crs')
    pr = reformat_data(pr, 'prec')

    state_data = xr.merge([pr, tmax, tmin])
    state_data.to_netcdf(cur_dir.joinpath('input',f'chinle_state_{year}.nc'))
    

Load basin shapefile for clipping

In [ ]:
gdf = gpd.read_file(cur_dir.joinpath('input','shp','Chinle_w_Mesa_Footprint.shp'))
gdf = gdf.to_crs(epsg=4326)

In [ ]:


for year in [2005, 2006]:

    met_data = process_forcing_data(cur_dir, year, gdf)

    domain_data = process_domain_data(cur_dir, met_data, gdf)

    state_data = process_state_data(cur_dir, year, gdf)

    drange = pd.date_range(f'{year}-01-01', f'{year}-12-31')

    params = {
        'time_step':60,
        'start':drange[0],
        'stop':drange[-1],
        'forcing': cur_dir.joinpath('input',f'chinle_forcing_{year}.nc').__str__(),
        'domain' : cur_dir.joinpath('input',f'chinle_domain.nc').__str__(),
        'state' : cur_dir.joinpath('input',f'chinle_state_{year}.nc').__str__(),
        'forcing_fmt':'netcdf',
        'out_dir':cur_dir.joinpath('output').__str__(),
        'out_prefix': f'chinle',
        'out_vars' : {'prec': {'out_name': 'prec', 'units': 'mm/hr'}},
        'chunks':{'lat':1,'lon':1},
        'forcing_vars': {'prec':'prec', 't_max': 't_max', 't_min': 't_min'},
        'domain_vars': {'elev': 'elev', 'lat': 'lat', 'lon': 'lon', 'mask': 'mask', 't_pk': 't_pk', 'dur': 'dur'},
        'prec_type': 'triangle',
        'scheduler': 'threading'
    }

    ms = MetSim(params)
    ms.run()

    del params, domain_data, state_data, met_data, ms, year

    



    


    